In [ ]:
import os
import cv2
import shutil
import base64
import requests
import numpy as np
from deepface import DeepFace
os.makedirs("output", exist_ok=True)

FORGE_INSTANCE_URL = "https://hfs1s0vb3480kc-1234.proxy.runpod.net" # "http://127.0.0.1:1234" / "https://*-1234.proxy.runpod.net"

In [ ]:
def list_files(dir_path):
    return [dir_path+os.path.relpath(os.path.join(root, f), dir_path) for root, _, files in os.walk(dir_path) for f in files]

def upscale_image(img_inp_path, img_out_path):

    with open(img_inp_path, "rb") as f_img_inp:
        img_inp_base64 = base64.b64encode(f_img_inp.read()).decode('utf-8')
    # img_h, img_w, _ = cv2.imread(img_inp_path).shape

    payload = {
        "resize_mode": 0,                  # Sets the resize mode: 0 to upscale by upscaling_resize amount, 1 to upscale up to upscaling_resize_h x upscaling_resize_w.
        "show_extras_results": False,      # Should the backend return the generated image?
        "gfpgan_visibility": 0.35,         # Sets the visibility of GFPGAN, values should be between 0 and 1.
        "codeformer_visibility": 0,        # Sets the visibility of CodeFormer, values should be between 0 and 1.
        "codeformer_weight": 0,            # Sets the weight of CodeFormer, values should be between 0 and 1.
        "upscaling_resize": 4,             # By how much to upscale the image, only used when resize_mode=0.
        "upscaling_resize_w": 512,         # Target width for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_resize_h": 512,         # Target height for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_crop": True,            # Should the upscaler crop the image to fit in the chosen size?
        "upscaler_1": "R-ESRGAN 4x+",      # The name of the main upscaler to use
        "upscaler_2": "None",              # The name of the secondary upscaler to use
        "extras_upscaler_2_visibility": 0, # Sets the visibility of secondary upscaler, values should be between 0 and 1.
        "upscale_first": False,            # Should the upscaler run before restoring faces?
        "image": img_inp_base64            # Image to work on, must be a Base64 string containing the image's data.
    }

    def request_forge_instance(payload, img_out_path):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/extra-single-image", json=payload) as req:
                with open(img_out_path, "wb") as f:
                    f.write(base64.b64decode(req.json()["image"]))
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"⚠️ req.json(): {req.json()}")
    request_forge_instance(payload, img_out_path)

def resize_image(img_inp_path, img_out_path, new_width=1080):
    img = cv2.imread(img_inp_path)
    h, w, _ = img.shape
    new_height = int(h * new_width / w)
    img_resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
    cv2.imwrite(img_out_path, img_resized)

def create_facemask(img_inp_path, img_keypoints_path, img_mask_path, expand_scale_factor=1.2):
    def get_ls_keypoints(img_inp_path, img_out_path, expand_scale_factor):
        def expand_polygon(ls_keypoints, expand_scale_factor):
            points = np.array(ls_keypoints, dtype=np.float32)                      # Convert keypoints to a NumPy array
            centroid = np.mean(points, axis=0)                                     # Calculate the centroid of the polygon
            expanded_points = centroid + (points - centroid) * expand_scale_factor # Translate points to origin (subtract centroid), scale, and translate back
            expanded_points = expanded_points.astype(np.int32)                     # Convert back to integer coordinates for OpenCV
            return expanded_points.tolist()
        # -----
        face_objs = []
        try:
            face_objs = DeepFace.extract_faces(img_inp_path, detector_backend='retinaface', align=True)
        except:
            pass
        # -----
        ls_keypoints = []
        for fce in face_objs:
            x,y,w,h = fce["facial_area"]["x"], fce["facial_area"]["y"], fce["facial_area"]["w"], fce["facial_area"]["h"]
            ls_keypoints.append([(x,y), (x,y+h), (x+w,y+h), (x+w,y)])
        ls_keypoints_expanded = [expand_polygon(e, expand_scale_factor) for e in ls_keypoints]
        # ----- Draw keypoints -----
        img = cv2.imread(img_inp_path)
        for i in range(len(ls_keypoints)):
            DRAW_THICKNESS = int(img.shape[1]/100)
            cv2.polylines(img, [np.array(ls_keypoints[i], dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=DRAW_THICKNESS)
            cv2.polylines(img, [np.array(ls_keypoints_expanded[i], dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=DRAW_THICKNESS)
        cv2.imwrite(img_out_path, img)
        # ----- ---- -----
        return ls_keypoints_expanded

    ls_keypoints = get_ls_keypoints(img_inp_path, img_keypoints_path, expand_scale_factor)
    h, w, _ = cv2.imread(img_inp_path).shape

    # ----- Create mask image ----
    img_mask = np.full((h, w, 3), 255, dtype=np.uint8) # Create a white mask (Default to inpaint all)
    if len(ls_keypoints) == 0:
        print(f"⚠️ > {img_inp_path} > No face detected")
    elif len(ls_keypoints) == 1:
        img_mask = np.full((h, w, 3), 0, dtype=np.uint8) # Create a black mask
        face_polygon = np.array(ls_keypoints[0], np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(img_mask, [face_polygon], (255, 255, 255)) # Fill the face polygon with white
    elif len(ls_keypoints) > 1:
        print(f"⚠️ > {img_inp_path} > Many faces detected")
    cv2.imwrite(img_mask_path, img_mask)

def sd_img2img(img_inp_path, img_msk_path=None, img_out_path="output/_test.jpg"):

    img_inp_base64 = None
    img_msk_base64 = None
    if img_inp_path:
        with open(img_inp_path, "rb") as f:
            img_inp_base64 = base64.b64encode(f.read()).decode('utf-8')
    if img_msk_path:
        with open(img_msk_path, "rb") as f:
            img_msk_base64 = base64.b64encode(f.read()).decode('utf-8')
    img_h, img_w, _ = cv2.imread(img_inp_path).shape

    sd_params = {
        "prompt": "masterpiece, best quality, 1girl, artist: bacun",
        "negative": "bad quality, worst quality, worst detail, artist name, text, signature, logo, watermark",
        "width": img_w,
        "height": img_h,
        "model": "WAI_V11",
        # "model": "MJX_V07",
        "steps": 20,
        "cfg": 5,
        "denoising": 0.5,
    }

    payload = {
        "prompt": sd_params["prompt"],
        "negative_prompt": sd_params["negative"],
        "styles": [],
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": "Euler a",
        "scheduler": "Karras",
        "batch_size": 1,
        "n_iter": 1,
        "steps": sd_params["steps"],
        "cfg_scale": sd_params["cfg"],
        "distilled_cfg_scale": 3.5,
        "width": sd_params["width"],
        "height": sd_params["height"],
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "eta": 0,
        "denoising_strength": sd_params["denoising"],
        "s_min_uncond": 0.0,
        "s_churn": 0.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "s_noise": 1.0,
        "override_settings": {"sd_model_checkpoint": sd_params["model"], "CLIP_stop_at_last_layers": 1},
        "override_settings_restore_afterwards": False,
        # "refiner_checkpoint": "string",
        "refiner_switch_at": 0,
        "disable_extra_networks": False,
        # "firstpass_image": "string",
        "comments": {},
        "init_images": [img_inp_base64],
        "resize_mode": 1,
        "image_cfg_scale": 1.5,
        "mask": img_msk_base64,
        "mask_blur_x": 4,
        "mask_blur_y": 4,
        "mask_blur": 4,
        "mask_round": True,
        "inpainting_fill": 1,
        "inpaint_full_res": 0,
        "inpaint_full_res_padding": 32,
        "inpainting_mask_invert": 0,
        "initial_noise_multiplier": 1.0,
        # "latent_mask": "string",
        # "force_task_id": "string",
        "hr_distilled_cfg": 3.5,
        "sampler_index": "Euler",
        "include_init_images": False,
        "script_name": None,
        "script_args": [],
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
        # "infotext": "string"
    }

    def request_forge_instance(payload, savepath):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/img2img", json=payload) as req:
                with open(savepath, "wb") as f:
                    f.write(base64.b64decode(req.json()["images"][0]))
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"⚠️ req.json(): {req.json()}")

    request_forge_instance(payload, img_out_path)

In [ ]:
ALL_PATHS_INPUT = list_files("input/")
for img_inp_path in ALL_PATHS_INPUT:
    try:




        PATH_INPUT = img_inp_path
        OUTPUT_DIR_NAME = os.path.splitext(os.path.basename(PATH_INPUT))[0] # "path/input/img_0.jpg" -> "img_0"
        os.makedirs(f"output/{OUTPUT_DIR_NAME}", exist_ok=True)
        PATH_UPSCALED   = f"output/{OUTPUT_DIR_NAME}/1_upscaled.jpg"
        PATH_KEYPOINTS  = f"output/{OUTPUT_DIR_NAME}/2_keypoints.jpg"
        PATH_FACEMASK   = f"output/{OUTPUT_DIR_NAME}/2_facemask.jpg"
        PATH_UPSCALED_RESIZED = f"output/{OUTPUT_DIR_NAME}/3_resized.jpg",
        PATH_FACEMASK_RESIZED = f"output/{OUTPUT_DIR_NAME}/3_facemask.jpg",
        PATH_OUTPUT =           f"output/{OUTPUT_DIR_NAME}_4_output_00001.jpg",

        shutil.copy2(PATH_INPUT, f"output/{OUTPUT_DIR_NAME}/0_input{os.path.splitext(PATH_INPUT)[1]}")
        upscale_image(PATH_INPUT, PATH_UPSCALED)
        create_facemask(PATH_UPSCALED, PATH_KEYPOINTS, PATH_FACEMASK, expand_scale_factor=1.5)
        resize_image(PATH_UPSCALED, PATH_UPSCALED_RESIZED, new_width=1080)
        resize_image(PATH_FACEMASK, PATH_FACEMASK_RESIZED, new_width=1080)
        sd_img2img(PATH_UPSCALED_RESIZED, PATH_FACEMASK_RESIZED, PATH_OUTPUT)




    except Exception as er:
        os.makedirs("input_failed", exist_ok=True)
        shutil.copy2(img_inp_path, "input_failed/")
        print(f"⚠️ > {img_inp_path} > Error: {er}")